## HFpy 코드 상세 설명

### 개요

- 지하개발을 위한 지반조사는 통상적으로 수직시추공을 통해 수행되며 초기지압 측정을 위한 수압파쇄시험 역시 수직시추공을 이용함. 이 방법은 세 방향의 주응력 중 하나의 방향을 수직으로 가정하고 시추공 종방향에 수직인 평면 상에서 다른 두 주응력을 구함.
- 근래 지하개발 목적이 다양해지면서 지하공간 형태나 현지암반의 균열대 분포에 따라 경사시추공을 이용한 삼차원 초기지압 측정이 요구되고 있음. 삼차원 응력 해석은 고려해야 할 변수가 많을 뿐만 아니라 현지 측정값과 계산값의 오차를 최소화하는 최적화 과정을 거쳐야 함(KIGAM 기본사업 단계보고서, 2021).
- 반복 계산을 수행해야 하므로 Fortran, Matlab으로 작성한 기존 코드가 있으나, 본 과제에서는 오픈소스로 손쉽게 작성 및 활용할 수 있도록 Python으로 재작성하였음.
- Python의 장점을 활용(Harris et al., 2020)하여 기존 코드보다 단순한 형태로 작성하였으며 결과 가시화도 개선하였음.
- 응력 계산에 사용된 식과 측정값 등은 Synn et al. (2015)를 참조하였음.

### 기존 데이터파일 활용

- 앞서 제작된 코드는 현장조사 정보가 있는 dat 파일을 읽어들여 계산 수행 및 그래프 생성하는 방식이었음.
- 본 코드도 기존 자료를 활용하기 위해 dat 파일을 지정하도록 구성하였음.
- dat 파일 양식은 아래와 같음.

```python
0.028	98	2.404					
0.05	0.035	0.02404	0.	0.	0.	0.	0.
24
1	133.3	20.5	136.5 	4.519 	274	0	
1	133.3	20.5	139.0 	5.920 	84	1	
1	133.3	20.5	149.5 	6.946 	8	28	
1	133.3	20.5	149.5 	6.946 	251	2	
1	133.3	20.5	153.0 	5.159 	198	50	
1	348.3	10.3	121.0 	4.008 	107	7	
1	348.3	10.3	121.0 	4.008 	89	3	
1	348.3	10.3	126.5 	3.620 	68	37	
1	348.3	10.3	129.0 	4.780 	115	9	
1	348.3	10.3	129.0 	4.780 	90	3	
1	348.3	10.3	134.5 	5.064 	170	14	
1	348.3	10.3	140.0 	4.329 	171	61	
1	348.3	10.3	140.0 	4.329 	16	57	
1	348.3	10.3	140.0 	4.329 	8	74	
1	0	0	138.5 	6.527 	343	10	
1	0	0	143.5 	7.249 	171	8	
1	0	0	143.5 	7.249 	238	8	
1	133.3	20.5	141.1 	5.120 	91	11	
1	133.3	20.5	160.3 	5.455 	93	15	
1	0	0	145.5 	4.741 	83	32	
1	0	0	149.0 	5.228 	277	4
1	348.3	10.3	144.0 	6.466 	164	4	
1	348.3	10.3	148.5 	4.571 	339	39	
1	348.3	10.3	151.5 	6.095 	21	11
	
1	133.3	20.5	158.5 	7.861 	321	5	
1	133.3	20.5	158.5 	7.861 	252	21	
1	348.3	10.3	121.0 	4.008 	353	11	
1	348.3	10.3	121.0 	4.008 	156	0	
1	0	0	138.5 	6.527 	100	3	
1	0	0	138.5 	6.527 	88	9
```


In [2]:
import math
import os
import sys
from tkinter import Tk
from tkinter.filedialog import askopenfilename

import numpy as np
import pandas as pd
import plotly.io as pio
import plotly.express as px
from scipy.optimize import minimize
from scipy.optimize import basinhopping
from scipy.optimize import least_squares